In [ ]:
import numpy as np
import xarray as xr

### Exploring Chloropyll A

Here we will focus on accessing and visualizing one global layer for chloropyll for just one monthly time step (which is its own `netcdf` file) representing January 2020. 

In [ ]:
# using locally downloaded dataset - on cholorphyll
ds = xr.open_dataset(
    "data/ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHLY_4km_GEO_PML_OCx-202001-fv5.0.nc"
)

In [ ]:
ds

In [ ]:
print(ds.dims)

In [ ]:
print(ds.data_vars)

In [ ]:
print(ds.variables.keys())

In [ ]:
# extracting only cholorphyll data
chlor_a = ds.variables["chlor_a"]
chlor_a

In [ ]:
chlor_a.attrs["units"]

In [ ]:
import matplotlib.pyplot as plt

chlor_a2d = chlor_a.isel(time=0)
chlor_a2d

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12.0, 8.0)

# Image is upside-down so we have to flip it here.
f = np.flip(chlor_a2d, 0)

plt.contourf(f, 20, cmap=plt.get_cmap("YlGnBu_r"))
plt.colorbar()
plt.show()

Note that this is an ocean based dataset - so we'd have to calculate values for each nation based on proximity - and only for those nations that touched the ocean. Additionally, we'll want to explore min/max values and be able to `zoom in` to particular regions as chloropyll A concentrations will accumulate only in those areas closer to the shore. 

### Exploring Land Surface Temperature

Now let's explore Land Surface Temperature, but this time let's try to access directly from the URL

In [ ]:
# url = ("https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc?download=1")


# original link above doesn't work, must add `#mode=bytes` to the end (see: https://github.com/Unidata/netcdf4-python/issues/1043)
url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc#mode=bytes"

In [ ]:
ds_disk = xr.open_dataset(url)
# ds_disk = xr.open_dataset(url, decode_times=False)
ds_disk

In [ ]:
# subset to bbox

In [ ]:
# Convert from Kelvin to Celsius
lst = ds_disk["lst"] - 273.15
lst

We'll visualize only 1 time step here - but it would be great to visualize the whole time series:
* we could visualize instead as a line graph
* or as visualization that loops over the entire time period 

But more importantly, we're going to have to aggregate these daily values to monthly. 

In [ ]:
lst.dims

In [ ]:
# selecting single time point (first time-step)
lst_2d = lst.isel(time=0)
lst_2d

In [ ]:
plt.contourf(lst_2d, 20, cmap=plt.get_cmap("coolwarm"))
plt.colorbar()
plt.show()

# Would prefer hvplot, but datashader not yet supported on python 3.11

In addition to aggregating these daily values into monthly means, we would also require that these calculatons were done for the admin level of interest (i.e., using zonal statistics). 

In [ ]:
import geopandas as gpd

admin0_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
# admin0_gdf

print(admin0_gdf.crs)

In [ ]:
print(lst.crs)

In [ ]:
lst = lst.set_crs(epsg=4326)
lst.crs

In [ ]:
bbox = admin0_gdf.geometry.total_bounds
print(bbox)

In [ ]:
# Subset to bounding box of African continent

african_subset = lst.rio.clip(bbox)

african_subset

In [ ]:
import pandas as pd
import rasterio as rio
import rasterstats as rs
from rasterstats import zonal_stats
import rioxarray

In [ ]:
# admin0_lst = rs.zonal_stats(
#     admin0_gdf,
#     # lst_2d.squeeze().values,
#     lst_2d.values,
#     categorical=True,
#     affine=lst_2d.rio.transform(),
#     stats="mean")